<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/EqBalancing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import pandas as pd
import numpy as np

In [63]:
rxn=r'aKNO3 + 5C12H22O11 -> bN2 + cCO2 + dH2O + eK2CO3'

In [64]:
rxn = re.sub(r'\s+','',rxn)
left_rxn,right_rxn = rxn.split('->')

In [65]:
stoic_pat=re.compile(r'(.*?)([A-Z].*)')
atom_pat=re.compile(r'([A-Z][a-z]?)(\d*)')

In [66]:
def process(side, sign):
    stoic_mols = side.split('+')
    mol_dict={}
    for stoic_mol in stoic_mols:
        res = stoic_pat.match(stoic_mol)
        stoic,mol=res[1],res[2]
        stoic = 1 if stoic=='' else (int(stoic)*sign if stoic.isnumeric() else stoic)
        atom_nums=atom_pat.findall(mol)
        d={'stoic':stoic}
        for atom,num in atom_nums:
            d[atom]=1 if num=='' else int(num)
        mol_dict[mol]=d
    df=pd.DataFrame(mol_dict)
    df=df.fillna(0)
    return df

df_left=process(left_rxn, -1)  
df_right=process(right_rxn, 1)
df=pd.concat([df_left,df_right],axis=1)

In [68]:
fixed_col=pd.to_numeric(df.loc['stoic'],errors='coerce').notna()
atom_num=df.iloc[1:,:].astype(int)

In [71]:
df

,KNO3,C12H22O11,N2,CO2,H2O,K2CO3
stoic,a,-5.0,b,c,d,e
K,1,0.0,0,0,0,2
N,1,0.0,2,0,0,0
O,3,11.0,0,2,1,3
C,0,12.0,0,1,0,1
H,0,22.0,0,0,2,0


In [72]:
rhs = - (df.loc['stoic',fixed_col].astype(int) * atom_num.loc[:,fixed_col]).to_numpy()
rhs = np.sum(rhs,axis=1)
solved_stoic=np.linalg.solve(atom_num.loc[:,~fixed_col],rhs)
df.loc['stoic',~fixed_col]=solved_stoic
df

,KNO3,C12H22O11,N2,CO2,H2O,K2CO3
stoic,-48.0,-5.0,24.0,36.0,55.0,24.0
K,1,0.0,0,0,0,2
N,1,0.0,2,0,0,0
O,3,11.0,0,2,1,3
C,0,12.0,0,1,0,1
H,0,22.0,0,0,2,0


In [86]:

reactant = df.loc['stoic',:]<0
nreactant = np.sum(reactant)
for i,(stoic, mol) in enumerate(zip(df.loc['stoic',reactant], df.columns[reactant])):
    print(int(abs(stoic)),end='')
    print(mol,end='')
    if i<nreactant-1:
        print(' + ', end='')
print(' -> ', end='')
prod = df.loc['stoic',:]>0
nprod = np.sum(prod)
for i,(stoic, mol) in enumerate(zip(df.loc['stoic',prod], df.columns[prod])):
    print(int(abs(stoic)),end='')
    print(mol,end='')
    if i<nprod-1:
        print(' + ', end='')

48KNO3 + 5C12H22O11 -> 24N2 + 36CO2 + 55H2O + 24K2CO3